<a href="https://colab.research.google.com/github/ThrishaJawahar/Falcon-9-Launch-Prediction/blob/main/VisualAnalyticswithFolium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objectives


1. Mark all launch sites on a map
2. Mark the success/failed launches for each site on the map
3. Calculate the distances between a launch site to its proximities

In [1]:
import folium
import pandas as pd

In [2]:

from folium.plugins import MarkerCluster

from folium.plugins import MousePosition

from folium.features import DivIcon

Task 1: Mark all launch sites on a map

In [3]:


launch_sites = {
    "CCAFS LC-40": [28.3922, -80.6077],
    "KSC LC-39B": [28.5721, -80.6480],
    "VAFB SLC-4E": [34.6328, -120.6108],
    "CCAFS SLC-40": [28.3922, -80.6077],

}


map_center = [28.3922, -80.6077]
map_obj = folium.Map(location=map_center, zoom_start=6)


for site, coordinates in launch_sites.items():
    folium.Marker(
        location=coordinates,
        popup=site,
        icon=folium.Icon(color="blue", icon="cloud"),
    ).add_to(map_obj)


map_obj


In [5]:
import pandas as pd
import requests
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

response = requests.get(URL)


if response.status_code == 200:

    spacex_csv_file = io.BytesIO(response.content)
    spacex_df = pd.read_csv(spacex_csv_file)


else:
    print(f"Error fetching data: Status code {response.status_code}")

In [6]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(
    nasa_coordinate,

    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:

launch_sites = {
    "CCAFS LC-40": [28.562302, -80.577356],
    "CCAFS SLC-40": [28.563197, -80.576820],
    "KSC LC-39A": [28.573255, -80.646895],
    "VAFB SLC-4E": [34.632834, -120.610745]
}


map_center = [28.573255, -80.646895]
site_map = folium.Map(location=map_center, zoom_start=6)


for site, coords in launch_sites.items():

    folium.Circle(
        location=coords,
        radius=500,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.3
    ).add_to(site_map)


    folium.Marker(
        location=coords,
        popup=site
    ).add_to(site_map)


site_map.save("launch_sites_map.html")


site_map


Task 2: Mark the success/failed launches for each site on the map


In [10]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

In [11]:
site_map = folium.Map(location=[28.562302, -80.577356], zoom_start=5)

marker_cluster = MarkerCluster().add_to(site_map)
for index, record in spacex_df.iterrows():

    marker_color = assign_marker_color(record['class'])

    folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=record['Launch Site'],
        icon=folium.Icon(color='white', icon_color=marker_color),
    ).add_to(marker_cluster)

site_map

In [12]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


TASK 3: Calculate the distances between a launch site to its proximities

In [17]:
from geopy.distance import geodesic

def calculate_distance(site1, site2):
    return geodesic(site1, site2).kilometers


launch_sites = {
    "CCAFS LC-40": [28.3922, -80.6077],
    "KSC LC-39B": [28.5721, -80.6480],
    "VAFB SLC-4E": [34.6328, -120.6108],
    "CCAFS SLC-40": [28.3922, -80.6077],
}


distance = calculate_distance(launch_sites["CCAFS LC-40"], launch_sites["KSC LC-39B"])
print(f"Distance between CCAFS LC-40 and KSC LC-39B: {distance:.2f} km")

map_obj


Distance between CCAFS LC-40 and KSC LC-39B: 20.32 km


In [18]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):

    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [21]:
from geopy.distance import geodesic

# Replace with your launch site coordinates
launch_site_coords = [28.562302, -80.577356]

# Replace with the coastline coordinates you obtained
coastline_coords = [28.563197, -80.576820]

distance = geodesic(launch_site_coords, coastline_coords).kilometers

print(f"Distance between launch site and coastline: {distance:.2f} km")

Distance between launch site and coastline: 0.11 km


In [22]:
launch_site_coords = [28.562302, -80.577356]
coastline_coords = [28.563197, -80.576820]
polyline = folium.PolyLine(locations=[launch_site_coords, coastline_coords], color='blue', weight=2.5)
site_map.add_child(polyline)